# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
# --- store the filepath in a variable ---
file = "../output_data/cities.csv"

# --- read the csv file and create a dataframe ---
cities_data = pd.DataFrame(pd.read_csv(file, index_col="City ID"))

# --- display the dataframe ---
cities_data


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# --- access maps with unique API key ---
gmaps.configure(api_key=g_key)

# --- use Lat and Lng as locations and Humidity as the weight ---
locations = cities_data[["Lat", "Lng"]]
weights = cities_data["Humidity (%)"]

# --- customize the size of the figure and add heatmap layer to map ---
figure_layout = {
    'width': "100%",
    'height': '400px',
#    'border': '1px solid black',
    'padding': '1px',
#    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center=(20.0, 5.0), zoom_level=1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False)
fig.add_layer(heatmap_layer)

# --- set maximum intensity and point_radius settings ---
heatmap_layer.max_intensity = max(weights)
heatmap_layer.point_radius = 3

# --- display heatmap ---
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# --- create dataframe with conditional statements to fit the below criteria ---
# 1. A max temperature lower than 80 degrees but higher than 70.
# 2. Wind speed less than 10 mph.
# 3. Zero cloudiness.
# 4. Humidity is less than 75%
# 5

ideal_cities = cities_data.loc[(cities_data["Max Temp (F)"] > 70) & 
                               (cities_data["Max Temp (F)"] < 80) & 
                               (cities_data["Wind Speed (mph)"] < 10) & 
                               (cities_data["Cloudiness (%)"] == 0) &
                               (cities_data["Humidity (%)"] < 75), :].reset_index(drop=True).dropna()
ideal_cities

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# --- store into variable named hotel_df ---

hotel_df = ideal_cities[["City", "Country", "Lat", "Lng"]].copy()

# --- Add a "Hotel Name" column to the DataFrame with null values ---
hotel_df["Hotel Name"] = np.nan
hotel_df


In [ ]:
# --- set up parameters for querying Google Places API ---
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# --- set up a parameters dictionary ---
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# --- define base url ---
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [ ]:

# --- use a for loop to go through each city in the dataframe and make an API call ---

for index, row in hotel_df.iterrows():
    
    # --- get the coordinates and store in params dictionary ---
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # --- call the API and get hotel details for each city ---
    hotel_data = requests.get(base_url, params).json()
    
    # --- use exception handling to save the first hotel name for each city ---
    
    try:
        hotel_name = hotel_data["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"City '{row['City']}' found! The nearest hotel is {hotel_name}")
        print("-------------------------------------------------------------------------")
              
    except:
        print(f"Could not find information for city '{row['City']}'... skipping.")    
        print("-------------------------------------------------------------------------")

In [ ]:
# --- remove null rows from dataframe ---
hotel_df.dropna(how="any",inplace=True)
hotel_df.reset_index(drop=True, inplace=True)

# --- display final dataframe ---
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# --- add marker layer on top of heat map ---
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig